In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept
import scipy

from loading_data import get_data
from utils_maze import get_trials

In [ ]:
thisdir = os.getcwd()
dataloc = os.path.join(thisdir, 'cache', 'data')
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "speed")

In [ ]:
import info.r068d8 as info
from run import spike_sorted_infos
infos = spike_sorted_infos

In [ ]:
for info in  infos:
    print(info.session_id)
    events, position, _, _, _ = get_data(info)
    speed = position.speed(t_smooth=1.)

    s = nept.find_nearest_idx(position.time, info.task_times["pauseB"].stop-30)
    e = nept.find_nearest_idx(position.time, info.task_times["phase3"].stop)

    runs = nept.run_threshold(position, thresh=15., t_smooth=1.)

    fig, ax = plt.subplots()
    ax.plot(position[s:e].time, position.x[s:e], ".", ms=3)
    plt.plot(speed.time, speed.data, color="r")
    for start, stop in zip(runs.starts, runs.stops):
        ax.fill_between([start, stop], np.max(position.y), color="k", alpha=0.2)
    plt.axhline(4, color="g")
    plt.axhline(10, color="b")
    plt.xlim(6500, 7000)
    plt.show()

In [ ]:
plt.figure()
plt.hist(speed.data, bins=[0, 2, 4, 6, 8, 10, 15, 20, 40, 60, 80, 200])
print(np.mean(spd.data))